In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [2]:
%%time
print('pickle read')
allwind = pd.read_pickle('brommer_raw_2020-10-04_2022-10-03.pkl')

pickle read
CPU times: user 64.5 ms, sys: 32.2 ms, total: 96.7 ms
Wall time: 109 ms


In [3]:
#%%time
#allwind = pd.read_csv('brommer_raw_2020-10-04_2022-10-03.csv')

In [4]:
allwind['Month'] = pd.DatetimeIndex(allwind['Date']).month_name().str[:3]

In [5]:
allwind.loc[(allwind.Time.str.startswith(("10:","11:")) & allwind.Time.str.endswith("PM")) |\
            (allwind.Time.str.startswith(("12:","1:","2:","3:","4:","5:")) & allwind.Time.str.endswith("AM")) ,\
            "Daypart"] = "Overnight"

allwind.loc[allwind.Time.str.startswith(("6:","7:","8:","9:")) & \
            allwind.Time.str.endswith("AM"), \
            "Daypart"] = "Morning"

allwind.loc[(allwind.Time.str.startswith(("10:","11:")) & allwind.Time.str.endswith("AM")) |\
            (allwind.Time.str.startswith(("12:","1:")) & allwind.Time.str.endswith("PM")) ,\
            "Daypart"] = "Midday"

allwind.loc[allwind.Time.str.startswith(("2:","3:","4:","5:")) & \
            allwind.Time.str.endswith("PM"), \
            "Daypart"] = "Afternoon"

allwind.loc[allwind.Time.str.startswith(("6:","7:","8:","9:")) & \
            allwind.Time.str.endswith("PM"), \
            "Daypart"] = "Evening"

In [6]:
allwind.loc[allwind.Month.isin(['Dec','Jan','Feb']) ,\
            "Season"] = "Winter"

allwind.loc[allwind.Month.isin(['Mar','Apr','May']) ,\
            "Season"] = "Spring"
             
allwind.loc[allwind.Month.isin(['Jun','Jul','Aug']) ,\
            "Season"] = "Summer"
             
allwind.loc[allwind.Month.isin(['Sep','Oct','Nov']) ,\
            "Season"] = "Fall"

In [7]:
allwind = allwind[['Date','Time','Season','Month','Daypart','Temp','Wind','Azimuth','Speed','Gust']]

In [8]:
allwind[::len(allwind)//10]

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust
0,2020-10-04,12:04 AM,Fall,Oct,Overnight,55.6,East,90.0,0.0,0.0
20405,2020-12-14,12:34 AM,Winter,Dec,Overnight,45.4,WSW,247.5,0.0,0.9
40810,2021-02-23,12:24 AM,Winter,Feb,Overnight,49.4,WSW,247.5,0.0,0.0
61215,2021-05-20,7:24 AM,Spring,May,Morning,47.6,NE,45.0,1.2,4.0
81620,2021-07-30,1:29 PM,Summer,Jul,Midday,63.9,SW,225.0,4.3,8.0
102025,2021-10-09,4:29 PM,Fall,Oct,Afternoon,67.8,WSW,247.5,3.0,8.0
122430,2021-12-19,9:52 PM,Winter,Dec,Evening,51.1,SSW,202.5,7.0,8.0
142835,2022-03-01,8:14 PM,Spring,Mar,Evening,54.9,East,90.0,0.9,6.0
163240,2022-05-12,12:24 AM,Spring,May,Overnight,47.7,NNE,22.5,2.4,4.0
183645,2022-07-23,12:19 AM,Summer,Jul,Overnight,56.2,West,270.0,0.9,3.0


In [9]:
allwind.to_csv('brommer_raw_2020-10-04_2022-10-03.csv')
allwind.to_pickle('brommer_raw_2020-10-04_2022-10-03.pkl')